In [1]:
import torch
from datasets import load_dataset
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType

/home/yinpu/miniconda3/envs/pytorch2.3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained("/home/yinpu/Projects/llm-tutorial/llm-embedding/Qwen/Qwen2.5-0.5B", trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained("/home/yinpu/Projects/llm-tutorial/llm-embedding/Qwen/Qwen2.5-0.5B", device_map="auto", trust_remote_code=True)

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1
)

model = get_peft_model(model, peft_config)

In [3]:
model_ref = AutoModelForCausalLM.from_pretrained("/home/yinpu/Projects/llm-tutorial/llm-embedding/Qwen/Qwen2.5-0.5B", device_map="auto", trust_remote_code=True)

In [4]:

from torch.utils.data import Dataset,DataLoader
class dpo_dataset(Dataset):
    def __init__(self,file,tokenizer,max_seq_length):
        self.tokenizer = tokenizer
        self.max_seq_length = max_seq_length
        # 打开json文件 用transformers
        self.data_list = load_dataset("json",data_files=file)['train']
    def __len__(self):
        return len(self.data_list)
    def __getitem__(self,index):
        # 取出data_list的一条数据  --> {"chosen":xxx,"rejected":xxx,"prompt":xxx} 一条数据是这样的格式
        data = self.data_list[index]

        # 对prompt reject和chosen进行tokenize  判断是否需要截断 保证所有的input_ids都一样 不够长度的直接padding  
        # 适配qwen 的 template  添加eos token
        prompt_input_ids = self.tokenizer.encode('<|im_start|>' + data['prompt'] + '<|im_end|>',add_special_tokens=False)
        chosen_input_ids = self.tokenizer.encode(data['chosen'],add_special_tokens=False)
        rejected_input_ids = self.tokenizer.encode(data['rejected'],add_special_tokens=False)

        prompt_input_ids = prompt_input_ids + [self.tokenizer.pad_token_id]
        # 设置labels
        chosen_labels = [-100] * len(prompt_input_ids) + chosen_input_ids + [self.tokenizer.pad_token_id]
        rejected_labels = [-100] * len(prompt_input_ids) + rejected_input_ids + [self.tokenizer.pad_token_id]
        chosen_input_ids = prompt_input_ids + chosen_input_ids + [self.tokenizer.pad_token_id]
        rejected_input_ids = prompt_input_ids + rejected_input_ids + [self.tokenizer.pad_token_id]

        assert len(chosen_labels) == len(chosen_input_ids)
        assert len(rejected_labels) == len(rejected_input_ids)

        inputs = dict(
            prompt_input_ids=prompt_input_ids,
            prompt_attention_mask=[1]*len(prompt_input_ids),
            chosen_input_ids=chosen_input_ids,
            chosen_attention_mask=[1]*len(chosen_input_ids),
            chosen_labels=chosen_labels,
            rejected_input_ids=rejected_input_ids,
            rejected_attention_mask=[1]*len(rejected_input_ids),
            rejected_labels=rejected_labels,
        )
        return inputs
    def map(self, func, **kwargs):
        return self

In [5]:
train_dataset = dpo_dataset(file = 'data.json', tokenizer = tokenizer, max_seq_length = 50)

In [9]:
from trl import DPOTrainer, DPOConfig

In [15]:
training_args = DPOConfig(
        num_train_epochs = 1,
        per_device_train_batch_size=2,
        learning_rate=3e-4,
        output_dir="./",
        save_total_limit = 1,
        logging_strategy = "steps",
        logging_steps = 50,
        seed = 103,
        fp16 = True,
        warmup_steps = 100,
    )

In [16]:
dpo_trainer = DPOTrainer(
        model,
        model_ref,
        beta=0.1, # DPO 的温度超参
        train_dataset=train_dataset, # 上文准备好的数据集
        tokenizer=tokenizer, # 分词器
        args=training_args)

/home/yinpu/miniconda3/envs/pytorch2.3/lib/python3.12/site-packages/trl/trainer/dpo_trainer.py:680: UserWarning: `max_length` is not set in the DPOConfig's init it will default to `512` by default, but you should do it yourself in the future.
  warnings.warn(
/home/yinpu/miniconda3/envs/pytorch2.3/lib/python3.12/site-packages/trl/trainer/dpo_trainer.py:693: UserWarning: `max_prompt_length` is not set in the DPOConfig's init it will default to `128` by default, but you should do it yourself in the future.
  warnings.warn(
/home/yinpu/miniconda3/envs/pytorch2.3/lib/python3.12/site-packages/trl/trainer/dpo_trainer.py:728: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


In [17]:
dpo_trainer.train()

100%|██████████| 12/12 [00:00<00:00, 13.01it/s]

{'train_runtime': 0.9267, 'train_samples_per_second': 24.819, 'train_steps_per_second': 12.949, 'train_loss': 0.6687090396881104, 'epoch': 1.0}


TrainOutput(global_step=12, training_loss=0.6687090396881104, metrics={'train_runtime': 0.9267, 'train_samples_per_second': 24.819, 'train_steps_per_second': 12.949, 'total_flos': 0.0, 'train_loss': 0.6687090396881104, 'epoch': 1.0})